In [82]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
# disable warnings
import warnings
warnings.filterwarnings('ignore')

# params 1
# nodes = ['1', '2', '3', '4', '5']
# send_edges = [('1', '2', 'a'),('2', '4', 'b'),('4', '5', 'c'),('3', '5', 'd')]
# receive_edges = [('2', '3', 'c')]

# params 2
nodes = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
send_edges = [
    ('1', '2', 'a'),
    ('2', '3', 'b'),
    ('4', '5', 'c'),
    ('5', '8', 'd'),
    ('5', '8', 'e'),
    ('5', '8', 'f')]
receive_edges = [('2', '4', 'b'),
                 ('4', '6', 'd'),
                 ('4', '7', 'e'),
                 ('6', '8', 'd'),
                 ('7', '8', 'e'),
                 ('8', '9', 'a'),
                 ]
all_edges = send_edges + receive_edges

In [83]:
def create_node_exp(node, node_strings, node_exptree):
    incoming_edges = [edge for edge in all_edges if edge[1] == node]
    incoming_nodes = np.unique([edge[0] for edge in incoming_edges])
    outgoing_edges = [edge for edge in all_edges if edge[0] == node]
    # 
    outgoing_sending_edges = [edge for edge in outgoing_edges if edge in send_edges]
    outgoing_receive_edges = [edge for edge in outgoing_edges if edge in receive_edges]
    # first, calculate incoming_str
    if len(incoming_edges) == 0:
        node_strings[node][0] = 'inf'
        node_exptree[node][0] = {
            'exp' : 'inf',
            'value': 'inf'
        }
    else:
        for in_node in incoming_nodes:
            if node_strings[node][0] == '':
                # node_exptree[node][0] = node_exptree[edge[0]][1][node]
                node_strings[node][0] = node_strings[in_node][1][node]
                node_exptree[node][0] = node_exptree[in_node][1]['sum'][node]
            else:
                # node_exptree[node][0] += " + " + node_exptree[edge[0]][1][node]
                node_strings[node][0] += " + " + node_strings[in_node][1][node]
                node_exptree[node][0] = {
                    'exp' : 'add',
                    'left': node_exptree[node][0],
                    'right': node_exptree[in_node][1]['sum'][node]
                }
    # second, calculate outgoing_str_to_edge for each outgoing edge
    if len(outgoing_edges) > 0:
        # create a list of tuples (node, outgoing_str_to_edge)
        outgoing_str_to_edge = {}
        outgoing_exptree_to_edge = {}
        for edge in outgoing_sending_edges:
            if edge[1] in outgoing_str_to_edge:
                # outgoing_str_to_edge[edge[1]] += f" + |w|_{edge[2]}"
                outgoing_str_to_edge[edge[1]] += f" + |w|_{edge[2]}"
                outgoing_exptree_to_edge[edge[1]] = {
                    'exp' : 'add',
                    'left': outgoing_exptree_to_edge[edge[1]],
                    'right': {
                        'exp': 'count',
                        'letter': edge[2]
                    }
                }
            else:
                # outgoing_str_to_edge[edge[1]] = f"|w|_{edge[2]}"
                outgoing_str_to_edge[edge[1]] = f"|w|_{edge[2]}"
                outgoing_exptree_to_edge[edge[1]] = {
                    'exp': 'count',
                    'letter': edge[2]
                }
        # find outgoing_str_to_edge for each outgoing receiving edge
        ouput_str = node_strings[node][0]
        ouput_exptree = node_exptree[node][0]
        for edge in outgoing_sending_edges:
            ouput_str += f" - {outgoing_str_to_edge[edge[1]]}"
            ouput_exptree = {
                'exp' : 'sub',
                'left': ouput_exptree,
                'right': outgoing_exptree_to_edge[edge[1]]
            }
            # ouput_str += f" - |w|_{edge[2]}"
        # if only one outgoing receive edge, then (input - senders_output)_t_{l}
        if len(outgoing_receive_edges) == 1:
            # add to output -1 if a sending is in outgoing_receive_edges
            if outgoing_receive_edges[0][2] in [edge[2] for edge in outgoing_sending_edges]:
                ouput_str += f" - 1"
                ouput_exptree = {
                    'exp' : 'sub',
                    'left': ouput_exptree,
                    'right': {
                        'exp': 'num',
                        'num': 1
                    }
                }
            # outgoing_str_to_edge[outgoing_receive_edges[0][1]] = f"({ouput_str})_to_{outgoing_receive_edges[0][2]}"
            outgoing_str_to_edge[outgoing_receive_edges[0][1]] = f"({ouput_str})_to_{outgoing_receive_edges[0][2]}"
            outgoing_exptree_to_edge[outgoing_receive_edges[0][1]] = {
                'exp' : 'to',
                'inner': ouput_exptree,
                'to': outgoing_receive_edges[0][2]
            }
        # if many outgoing receive edges, then (input - senders_output)_section_{l}_{all_receive_letters}
        elif len(outgoing_receive_edges) > 1:
            # join all letters in outgoing_receive_edges to one string named letters
            letters = "".join([edge[2] for edge in outgoing_receive_edges])
            for edge in outgoing_receive_edges:
                cur_ouput_str = ouput_str
                cur_ouput_exptree = ouput_exptree
                # add to output -1 if a sending is in outgoing_receive_edges
                if edge[2] in [e[2] for e in outgoing_sending_edges]:
                    cur_ouput_str += f" - 1"
                    cur_ouput_exptree = {
                        'exp' : 'sub',
                        'left': cur_ouput_exptree,
                        'right': {
                            'exp': 'num',
                            'num': 1
                        }
                    }
                #
                if edge[1] in outgoing_str_to_edge:
                    # outgoing_str_to_edge[edge[1]] += f" + ({ouput_str})_section_{edge[2]}_({letters})"
                    outgoing_str_to_edge[edge[1]] += f" + ({cur_ouput_str})_section_{edge[2]}_({letters})"
                    outgoing_exptree_to_edge[edge[1]] = {
                        'exp' : 'add',
                        'left': outgoing_exptree_to_edge[edge[1]],
                        'right': {
                            'exp': 'section',
                            'inner': cur_ouput_exptree,
                            'section': edge[2],
                            'letters': letters
                        }
                    }
                else:
                    # outgoing_str_to_edge[edge[1]] = f"({ouput_str})_section_{edge[2]}_({letters})"
                    outgoing_str_to_edge[edge[1]] = f"({cur_ouput_str})_section_{edge[2]}_({letters})"
                    outgoing_exptree_to_edge[edge[1]] = {
                        'exp': 'section',
                        'inner': cur_ouput_exptree,
                        'section': edge[2],
                        'letters': letters
                    }
        # add to node_strings
        node_strings[node][1] = outgoing_str_to_edge
        # add to node_exptree as array
        node_exptree[node][1] = {
            'exp': 'sum',
            'sum': outgoing_exptree_to_edge
        }
        # node_exptree[node][2] as joined outgoing_str_to_edge
        node_strings[node][2] = " + ".join([outgoing_str_to_edge[key] for key in outgoing_str_to_edge])
    # third, calculate remaining_str, simply (input - senders_output)
    node_strings[node][3] = f"{node_strings[node][0]} - ({node_strings[node][2]})" if node_exptree[node][0] != 'inf' else 'inf'
    node_exptree[node][2] = {
        'exp' : 'sub',
        'left': node_exptree[node][0],
        'right': node_exptree[node][1]
    }
    return node_strings, node_exptree


In [84]:

def node_exp_sequencial():
    # have a tuple for each node, init node: (incoming_str, outgoing_str_to_edge, remaining_str)
    node_exptree = {}
    node_strings = {}
    for node in nodes:
        node_exptree[node] = [{}, {}, {}] # incoming_str, outgoing_str_to_edge, outgoing_str_to_edge joined, remaining_str_with_incoming
        node_strings[node] = ['', {}, '', ''] # incoming_str, outgoing_str_to_edge, outgoing_str_to_edge joined, remaining_str_with_incoming

    # first: map sequance of nodes (note: no internal loops needed)
    all_edges = send_edges + receive_edges
    nodes_tier = []
    passed_nodes = []
    left_nodes = nodes.copy()
    node_tier = 0
    while len(passed_nodes) < len(nodes):
        nodes_tier.append([])
        for node in left_nodes:
            incoming_edges = [edge for edge in all_edges if edge[1] == node]
            # if incoming_edges subgroup in passed_nodes then add to nodes_tier[node_tier]
            if len([edge for edge in incoming_edges if edge[0] in passed_nodes]) == len(incoming_edges):
                nodes_tier[node_tier] = nodes_tier[node_tier] + [node]
        passed_nodes = passed_nodes + nodes_tier[node_tier]
        left_nodes = [node for node in nodes if node not in passed_nodes]
        node_tier += 1
    #
    while len(nodes_tier) > 0:
        for node in nodes_tier[0]:    
            node_strings, node_exptree = create_node_exp(node, node_strings, node_exptree)
            nodes_tier = nodes_tier[1:]
    return node_exptree, node_strings



In [85]:
node_exptree, node_strings = node_exp_sequencial()

# add all to dataframe
df = pd.DataFrame(columns=['node', 'incoming', 'outgoing', 'remaining'])
for node in nodes:
    df = df.append({'node': node, 'incoming': node_strings[node][0], 'outgoing': node_strings[node][2], 'remaining': node_strings[node][3]}, ignore_index=True)

print(df.to_string())

# # print object in json format
# print(json.dumps(node_exptree, indent=4))


{'1': ['', {}, '', ''], '2': ['', {}, '', ''], '3': ['', {}, '', ''], '4': ['', {}, '', ''], '5': ['', {}, '', ''], '6': ['', {}, '', ''], '7': ['', {}, '', ''], '8': ['', {}, '', ''], '9': ['', {}, '', '']}


KeyError: '8'

In [ ]:
def calculate_sum_of_letter(word, letter):
    return len([l for l in word if l == letter])

def cut_word_to_letter(word, letter):
    # get last index of letter in word
    last_index = word.rfind(letter)
    # if letter not in word, return empty string
    if last_index == -1:
        return word
    # else return until last index
    return word[:last_index]

def calculate_section(word, letter, letters):
    # cut word to sections by letters
    sections = []
    working_section = ''
    for l in word:
        if l in letters:
            if l == letter:
                sections.append(working_section)
            working_section = ''
        else:
            working_section = working_section + l
    # return merged sections
    return "".join(sections)

def eval_exptree(exp, word):
    if len(word) == 0:
        return 0
    if exp:
        if exp['exp'] == 'inf':
            return float('inf')
        elif exp['exp'] == 'add':
            return eval_exptree(exp['left'], word) + eval_exptree(exp['right'], word)
        elif exp['exp'] == 'sub':
            return eval_exptree(exp['left'], word) - eval_exptree(exp['right'], word)
        elif exp['exp'] == 'to':
            return eval_exptree(exp['inner'], cut_word_to_letter(word, exp['to']))
        elif exp['exp'] == 'section':
            return eval_exptree(exp['inner'], calculate_section(word, exp['section'], exp['letters']))
        elif exp['exp'] == 'sum':
            return sum([eval_exptree(exp['sum'][key], word) for key in exp['sum']])
        elif exp['exp'] == 'num':
            return exp['num']
        elif exp['exp'] == 'count':
            return calculate_sum_of_letter(word, exp['letter'])
        else:
            raise Exception('Unknown expression type ' + exp['exp'])
    else:
        return 0

def verifier(word, node_exptree):
    # calculate for each node number based on cur_word and remaining_str
    node_values = {}
    for node in node_exptree:
        node_values[node] = eval_exptree(node_exptree[node][2], word)
    return node_values

def debug_exp(exp, word):
    if len(word) == 0:
        return '~0'
    if exp:
        # print(exp)
        if exp['exp'] == 'inf':
            return 'inf'
        elif exp['exp'] == 'add':
            return f"({debug_exp(exp['left'], word)} + {debug_exp(exp['right'], word)})"
        elif exp['exp'] == 'sub':
            return f"({debug_exp(exp['left'], word)} - {debug_exp(exp['right'], word)})"
        elif exp['exp'] == 'to':
            return f"{debug_exp(exp['inner'], cut_word_to_letter(word, exp['to']))}"
        elif exp['exp'] == 'section':
            return f"{debug_exp(exp['inner'], calculate_section(word, exp['section'], exp['letters']))}"
        elif exp['exp'] == 'sum':
            return f"({' + '.join([debug_exp(exp['sum'][key], word) for key in exp['sum']])})"
        elif exp['exp'] == 'num':
            return str(exp['num'])
        elif exp['exp'] == 'count':
            return f"{calculate_sum_of_letter(word, exp['letter'])}"
        else:
            raise Exception('Unknown expression type ' + exp['exp'])
    else:
        return '0'
    
    
print(verifier('aaaabcdaaab', node_exptree))
# print(debug_exp(node_exptree['9'][2], 'aaaabcdaaab'))
# print(verifier('aaaabcdaaab', node_exptree)['4'])
# print(json.dumps(node_exptree['9'][2], indent=4))

{'1': inf, '2': 0, '3': 2, '4': 2, '5': 0, '6': 2, '7': 0, '8': 0, '9': 1}


In [ ]:
# simulate word over the BP
def simulate(word, nodes, send_edges, receive_edges):
    # init node_values
    node_values = {}
    for node in nodes:
        node_values[node] = [0, 0, 0] # [initial, in, out]
    node_values[nodes[0]] = [float('inf'), 0, 0]
    # for each letter in word
    for letter in word:
        # first, deal with sending edge
        for edge in send_edges:
            # if letter in edge, add to out
            if letter == edge[2]:
                if node_values[edge[0]][0] == 0:
                    raise Exception('Node ' + edge[0] + ' has no cores')
                node_values[edge[0]][0] -= 1 # deal with initial not out since sending overrules receiving
                node_values[edge[1]][1] += 1
                break
        # second, deal with receiving edges
        for edge in receive_edges:
            # if letter in edge, add to in
            if letter == edge[2]:
                node_values[edge[0]][2] = node_values[edge[0]][0]
                node_values[edge[1]][1] += node_values[edge[0]][0]
        # commit all in and out
        for node in nodes:
            node_values[node][0] = node_values[node][0] + node_values[node][1] - node_values[node][2]
            node_values[node][1] = 0
            node_values[node][2] = 0
    # return node_values first value in each node
    # convert to [(node, node_values[node][0]) for node in nodes] to dict
    return dict([(node, node_values[node][0]) for node in nodes])

# aaaabdcaaabe
print(simulate('aaaabcdaaab', nodes, send_edges, receive_edges))

{'1': inf, '2': 0, '3': 2, '4': 2, '5': 0, '6': 2, '7': 0, '8': 0, '9': 1}


In [ ]:
def compare_outputs(word, nodes, send_edges, receive_edges):
    #
    
    # get verifier output
    verifier_output = verifier(word, node_exptree)
    # get simulate output
    simulate_output = simulate(word, nodes, send_edges, receive_edges)
    # compare outputs
    for node in nodes:
        if verifier_output[node] != simulate_output[node]:
            raise Exception(f"Node {node} verifier output {verifier_output[node]} != simulate output {simulate_output[node]}")
    print('Outputs are equal')